In [1]:
from gklearn.utils.graphfiles import loadDataset
import networkx as nx
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from __future__ import print_function
import torch

In [3]:
def label_to_color(label):
    if label == 'C':
        return 0.1
    elif label == 'O':
        return 0.8
    
def nodes_to_color_sequence(G):
    return [label_to_color(c[1]['label'][0]) for c in G.nodes(data=True)]

Gs,y = loadDataset('../DeepGED/MAO/dataset.ds')
#for e in Gs[13].edges():
#    print(Gs[13][e[0]][e[1]]['bond_type'])
print('edge max label',max(max([[G[e[0]][e[1]]['bond_type'] for e in G.edges()] for G in Gs])))
G1 = Gs[1]
G2 = Gs[9]
print(y[1],y[9])
plt.figure(0)
nx.draw_networkx(G1,with_labels=True,node_color = nodes_to_color_sequence(G1),cmap='autumn')

plt.figure(1)
nx.draw_networkx(G2,with_labels=True,node_color = nodes_to_color_sequence(G2),cmap='autumn')

plt.show()
import extended_label
for g in Gs:
    extended_label.compute_extended_labels(g)
#for v in Gs[10].nodes():
#        print(Gs[10].nodes[v])

#print(nx.to_dict_of_lists(Gs[13]))



#dict={'C':0,'N':1,'O':2}
#A,labels=from_networkx_to_tensor2(Gs[13],dict)
#print(A)
#A1=(A==torch.ones(13,13)).int()
#A2=(A==2*torch.ones(13,13)).int()
#print(A1)
#print(A2)


edge max label 3
0.0 0.0


In [4]:
import torch 
print(torch.__version__)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
#model.to(device)

print(torch.cuda.device(0))
print(torch.cuda.device_count())


1.6.0
0


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import svd
import rings
from svd import iterated_power as compute_major_axis

#torch.autograd.set_detect_anomaly(True)

class Net(nn.Module):
    
        
    def __init__(self,GraphList,normalize=False,node_label='label'):
        super(Net, self).__init__()   
        self.normalize=normalize
        self.node_label=node_label
        dict,self.nb_edge_labels=self.build_node_dictionnary(GraphList)
        self.nb_labels=len(dict)
        print('nb_edge_labels : ', self.nb_edge_labels)
        self.device='cuda' if torch.cuda.is_available() else 'cpu'  #torch.device("cuda:0")
        nb_node_pair_label=self.nb_labels*(self.nb_labels-1)/2.0
        nb_edge_pair_label=int(self.nb_edge_labels*(self.nb_edge_labels-1)/2)
        
        self.node_weighs=nn.Parameter(torch.tensor(1.0/(nb_node_pair_label+nb_edge_pair_label+2))+(1e-3)*torch.rand(int(self.nb_labels*(self.nb_labels-1)/2+1),requires_grad=True,device=self.device)) # all substitution costs+ nodeIns/Del. old version: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del        
        self.edge_weighs=nn.Parameter(torch.tensor(1.0/(nb_node_pair_label+nb_edge_pair_label+2))+(1e-3)*torch.rand(nb_edge_pair_label+1,requires_grad=True,device=self.device)) #edgeIns/Del
                
        self.card=torch.tensor([G.order() for G in GraphList]).to(self.device)
        card_max=self.card.max()
        self.A=torch.empty((len(GraphList),card_max*card_max),dtype=torch.int,device=self.device)
        self.labels=torch.empty((len(GraphList),card_max),dtype=torch.int,device=self.device)
        print(self.A.shape)
        for k in range(len(GraphList)):
            A,l=self.from_networkx_to_tensor(GraphList[k],dict)             
            self.A[k,0:A.shape[1]]=A[0]
            self.labels[k,0:l.shape[0]]=l
        print('adjacency matrices',self.A)
        print('node labels',self.labels)
        print('order of the graphs',self.card)
        
    def forward(self, input):        
        ged=torch.zeros(len(input)).to(self.device) 
        node_costs,nodeInsDel,edge_costs,edgeInsDel=self.from_weighs_to_costs()
        
        #print('weighs:',self.weighs.device,'device:',self.device,'card:',self.card.device,'A:',self.A.device,'labels:',self.labels.device)
        for k in range(len(input)):            
            g1=input[k][0]
            g2=input[k][1]
            n=self.card[g1]
            m=self.card[g2]
            
            self.ring_g,self.ring_h = rings.build_rings(g1,edgeInsDel.size()), rings.build_rings(g2,edgeInsDel.size()) 
                        
            C=self.construct_cost_matrix(g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)      
            #S=self.mapping_from_similarity(C,n,m)
            #S=self.mapping_from_cost(C,n,m)
            
            #S=self.new_mapping_from_cost(C,n,m,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)
            S=self.mapping_from_cost_sans_FW(n,m,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)
            v=torch.flatten(S)
            
            normalize_factor=1.0
            if self.normalize:
                nb_edge1=(self.A[g1][0:n*n] != torch.zeros(n*n,device=self.device)).int().sum()
                nb_edge2=(self.A[g2][0:m*m] != torch.zeros(m*m,device=self.device)).int().sum()
                normalize_factor=nodeInsDel*(n+m)+edgeInsDel*(nb_edge1+nb_edge2)
                
            ged[k]=.5*(v.t()@C@v)/normalize_factor
        return ged
    
    def from_weighs_to_costs(self):
        
        #cn=torch.exp(self.node_weighs)
        #ce=torch.exp(self.edge_weighs)
        cn=self.node_weighs*self.node_weighs
        ce=self.edge_weighs*self.edge_weighs
        total_cost=cn.sum()+ce.sum()
        cn=cn/total_cost
        ce=ce/total_cost
        edgeInsDel=ce[-1]

        node_costs=torch.zeros((self.nb_labels,self.nb_labels),device=self.device)
        upper_part=torch.triu_indices(node_costs.shape[0],node_costs.shape[1],offset=1,device=self.device)        
        node_costs[upper_part[0],upper_part[1]]=cn[0:-1]
        node_costs=node_costs+node_costs.T

        if self.nb_edge_labels>1:
            edge_costs=torch.zeros((self.nb_edge_labels,self.nb_edge_labels),device=self.device)
            upper_part=torch.triu_indices(edge_costs.shape[0],edge_costs.shape[1],offset=1,device=self.device)        
            edge_costs[upper_part[0],upper_part[1]]=ce[0:-1]
            edge_costs=edge_costs+edge_costs.T
        else:
            edge_costs=torch.zeros(0,device=self.device)
        
        #print('res from_w_to_cost : ', node_costs,cn[-1],edge_costs,edgeInsDel)
        return node_costs,cn[-1],edge_costs,edgeInsDel
    
    def build_node_dictionnary(self,GraphList):
        #extraction de tous les labels d'atomes
        node_labels=[]
        for G in Gs:
            for v in nx.nodes(G):
                if not G.nodes[v][self.node_label][0] in node_labels:
                    node_labels.append(G.nodes[v][self.node_label][0])
        node_labels.sort()
        #extraction d'un dictionnaire permettant de numéroter chaque label par un numéro.
        dict={}
        k=0
        for label in node_labels:
            dict[label]=k
            k=k+1
        print(node_labels)
        print(dict,len(dict))
    
        return dict,max(max([[int(G[e[0]][e[1]]['bond_type']) for e in G.edges()] for G in GraphList]))
    
    def from_networkx_to_tensor(self,G,dict):    
        A=torch.tensor(nx.to_scipy_sparse_matrix(G,dtype=int,weight='bond_type').todense(),dtype=torch.int)        
        lab=[dict[G.nodes[v][self.node_label][0]] for v in nx.nodes(G)]
   
        return (A.view(1,A.shape[0]*A.shape[1]),torch.tensor(lab))

    def construct_cost_matrix(self,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel):
        n = self.card[g1].item()
        m = self.card[g2].item()
        
        A1=torch.zeros((n+1,n+1),dtype=torch.int,device=self.device)
        A1[0:n,0:n]=self.A[g1][0:n*n].view(n,n)
        A2=torch.zeros((m+1,m+1),dtype=torch.int,device=self.device)
        A2[0:m,0:m]=self.A[g2][0:m*m].view(m,m)
        
        
         # costs: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del
        
        #C=cost[3]*torch.cat([torch.cat([C12[l][k] for k in range(n+1)],1) for l in range(n+1)])
        #Pas bien sur mais cela semble fonctionner.
        C=edgeInsDel*self.matrix_edgeInsDel(A1,A2)
        if self.nb_edge_labels>1:
            for k in range(self.nb_edge_labels):
                for l in range(self.nb_edge_labels):
                    if k != l:
#                    C.add_(self.matrix_edgeSubst(A1,A2,k+1,l+1),alpha=edge_costs[k][l])
                        C=C+edge_costs[k][l]*self.matrix_edgeSubst(A1,A2,k+1,l+1)
        #C=cost[3]*torch.tensor(np.array([ [  k!=l and A1[k//(m+1),l//(m+1)]^A2[k%(m+1),l%(m+1)] for k in range((n+1)*(m+1))] for l in range((n+1)*(m+1))]),device=self.device)        

        l1=self.labels[g1][0:n]
        l2=self.labels[g2][0:m]
        D=torch.zeros((n+1)*(m+1),device=self.device)
        D[n*(m+1):]=nodeInsDel
        D[n*(m+1)+m]=0
        D[[i*(m+1)+m for i in range(n)]]=nodeInsDel
        D[[k for k in range(n*(m+1)) if k%(m+1) != m]]=torch.tensor([node_costs[l1[k//(m+1)],l2[k%(m+1)]] for k in range(n*(m+1)) if k%(m+1) != m],device=self.device )
        mask = torch.diag(torch.ones_like(D))
        C=mask*torch.diag(D) + (1. - mask)*C
        
        #C[range(len(C)),range(len(C))]=D
      
        return C
    
    def matrix_edgeInsDel(self,A1,A2):
        Abin1=(A1!=torch.zeros((A1.shape[0],A1.shape[1]),device=self.device))
        Abin2=(A2!=torch.zeros((A2.shape[0],A2.shape[1]),device=self.device))
        C1=torch.einsum('ij,kl->ijkl',torch.logical_not(Abin1),Abin2)
        C2=torch.einsum('ij,kl->ijkl',Abin1,torch.logical_not(Abin2))
        C12=torch.logical_or(C1,C2).int()
    
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C12,1),1),0),1)

    def matrix_edgeSubst(self,A1,A2,lab1,lab2):
        Abin1=(A1==lab1*torch.ones((A1.shape[0],A1.shape[1]),device=self.device)).int()
        Abin2=(A2==lab2*torch.ones((A2.shape[0],A2.shape[1]),device=self.device)).int()
        C=torch.einsum('ij,kl->ijkl',Abin1,Abin2)
        
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C,1),1),0),1)
    
    def similarity_from_cost(self,C):
        N=C.shape[0]
             
        #return (torch.norm(C,p='fro')*torch.eye(N,device=self.device) -C)
        return (C.max()*torch.eye(N,device=self.device) -C)
    

    
    def lsape_populate_instance(self,first_graph,second_graph,average_node_cost, average_edge_cost,alpha,lbda):       #ring_g, ring_h come from global ring with all graphs in so ring_g = rings['g'] and ring_h = rings['h']
        g,h = Gs[first_graph], Gs[second_graph]
        self.average_cost =[average_node_cost, average_edge_cost]
        self.first_graph, self.second_graph = first_graph,second_graph
        
        node_costs,nodeInsDel,edge_costs,edgeInsDel=self.from_weighs_to_costs()

        lsape_instance = [[0 for _ in range(len(g) + 1)] for __ in range(len(h) + 1)]
        for g_node_index in range(len(g) + 1):
            for h_node_index in range(len(h) + 1):
                lsape_instance[h_node_index][g_node_index] = rings.compute_ring_distance(g,h,self.ring_g,self.ring_h,g_node_index,h_node_index,alpha,lbda,node_costs,nodeInsDel,edge_costs,edgeInsDel,first_graph,second_graph)
        for i in lsape_instance :
            i = torch.as_tensor(i)
        lsape_instance = torch.as_tensor(lsape_instance)
        #print(type(lsape_instance))
        return lsape_instance
    
  
    def mapping_from_cost_sans_FW(self,n,m,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel): 
        c_0 =self.lsape_populate_instance(g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)
        x0=svd.eps_assigment_from_mapping(torch.exp(-c_0),10).view((n+1)*(m+1),1)
        return x0
    
    def new_mapping_from_cost(self,C,n,m,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel): 
        c=torch.diag(C)       
        c_0 =self.lsape_populate_instance(g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)
        D=C-torch.eye(C.shape[0],device=self.device)*c
        x0=svd.eps_assigment_from_mapping(torch.exp(-c_0),10).view((n+1)*(m+1),1)
        return svd.franck_wolfe(x0,D,c,5,15,n,m)
    
    def mapping_from_cost(self,C,n,m): #à améliorer car on prend que les coût des sommets
        c=torch.diag(C)       #diag donc vecteur/ matrice ligne, alors comment .view(n+1,m+1) est sensé marcher ?
        c_0=lsape_populate_instance(g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)
        print('C.shape[0] : ', C.shape[0])
        D=C-torch.eye(C.shape[0],device=self.device)*c
        print('c : ',c)
        #x0=svd.eps_assigment_from_mapping(c,10)
        x0=svd.eps_assigment_from_mapping(torch.exp(-c_0),10).view((n+1)*(m+1),1)
        return svd.franck_wolfe(x0,D,c,5,15,n,m)
    
        
    def mapping_from_similarity(self,C,n,m):
        M=self.similarity_from_cost(C)
        first_ev=compute_major_axis(M)
        #first_ev=self.iterated_power(M,inv=True)
        if(first_ev.sum() <0):
            first_ev=-first_ev
        S=torch.exp(first_ev.view(n+1,m+1)) # enforce the difference, accelerate the convergence. 
        S=self.eps_assigment_from_mapping(S)
        return  S
  
print('Gs=',len(Gs))
model = Net(Gs,normalize=True,node_label='extended_label')

#params = list(model.parameters())
#print(len(params))
#print(params[0])
#print(model(input))
print(max([G.order() for G in Gs]),len(Gs))
print('toto')

Gs= 68
['C_1C', 'C_1C1C1N', 'C_1C1C2C', 'C_1C1N', 'C_1C1N2C', 'C_1C1O', 'C_1C1O2C', 'C_1C2C', 'C_1C3C', 'C_1N', 'C_1O', 'C_2C', 'C_2C2C', 'C_3C', 'N_1C', 'N_1C1C', 'N_1C1C1C', 'O_1C', 'O_1C1C']
{'C_1C': 0, 'C_1C1C1N': 1, 'C_1C1C2C': 2, 'C_1C1N': 3, 'C_1C1N2C': 4, 'C_1C1O': 5, 'C_1C1O2C': 6, 'C_1C2C': 7, 'C_1C3C': 8, 'C_1N': 9, 'C_1O': 10, 'C_2C': 11, 'C_2C2C': 12, 'C_3C': 13, 'N_1C': 14, 'N_1C1C': 15, 'N_1C1C1C': 16, 'O_1C': 17, 'O_1C1C': 18} 19
nb_edge_labels :  3
torch.Size([68, 729])
adjacency matrices tensor([[         0,          1,          0,  ..., 1734632748, 1868783461,
          745763955],
        [         0,          1,          0,  ..., 1818575987,  537541161,
          538976288],
        [         0,          1,          0,  ...,  544106784, 1914728307,
         1600613993],
        ...,
        [         0,          1,          0,  ...,  538976288,  538976288,
         1126178848],
        [         0,          1,          0,  ...,       9504,          0,
             

In [6]:
import itertools
nb=len(Gs)
class1=torch.tensor([k for k in range(len(y)) if y[k]==1])
class2=torch.tensor([k for k in range(len(y)) if y[k]==0])

train_size=15
test_size=7

if train_size % 2 == 0:
    nb_class1=int(train_size/2)
    nb_class2=int(train_size/2)
else:
    nb_class1=int(train_size/2)+1
    nb_class2=int(train_size/2)
    
if test_size % 2 == 0:
    nb_class1_test=int(test_size/2)
    nb_class2_test=int(test_size/2)
else:
    nb_class1_test=int(test_size/2)+1
    nb_class2_test=int(test_size/2)
    
print((torch.abs(10000*torch.randn(nb_class1)).int()%class1.size()[0]).long())
random_class1=class1[(torch.abs(10000*torch.randn(nb_class1)).int()%class1.size()[0]).long()]
random_class2=class2[(torch.abs(10000*torch.randn(nb_class2)).int()%class2.size()[0]).long()]

random_class1_test=class1[(torch.abs(10000*torch.randn(nb_class1_test)).int()%class1.size()[0]).long()]
random_class2_test=class2[(torch.abs(10000*torch.randn(nb_class2_test)).int()%class2.size()[0]).long()]

train_graphs=torch.cat((random_class1,random_class2),0)
test_graphs=torch.cat((random_class1_test,random_class2_test),0)

print(train_graphs)
print(test_graphs)

couples=torch.triu_indices(train_size,train_size,offset=1)



#combinations=itertools.combinations(range(nb),2)

nb_elt=int(train_size*(train_size-1)/2)
data=torch.empty((nb_elt,2),dtype=torch.int)
yt=torch.ones(nb_elt)

data[0:nb_elt,0]=train_graphs[couples[0]]
data[0:nb_elt,1]=train_graphs[couples[1]]
print(nb_elt)
#couples=[]
for k in range(nb_elt):
    if (y[data[k][0]]!=y[data[k][1]]):
        yt[k]=-1.0        

#print('data=',data)

#print(couples[1:2])

torch.cuda.empty_cache()
if torch.cuda.is_available():
    device = torch.device("cuda:0")          # a CUDA device object


tensor([22,  9,  2,  6,  3,  7, 10, 14])
tensor([ 4, 66, 16, 60, 65, 10, 23, 45, 30, 57, 48, 55, 29, 40, 15])
tensor([53, 23, 36, 53, 51,  1, 31])
105


In [7]:
#if torch.cuda.device_count() > 1:
#  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
#  model = nn.DataParallel(model)
from triangular_losses import TriangularConstraint as triangular_constraint
from sklearn.neighbors import KNeighborsClassifier

model.to(device)

def classification(model,data,yt,nb_iter):

    min_loss,iter_min_loss=10,0
    criterion = torch.nn.HingeEmbeddingLoss(margin=1.0)
    criterionTri=triangular_constraint()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    print(device)

#torch.cat((same_class[0:20],diff_class[0:20]),0).to(device)
    input=data.to(device)
    target=yt.to(device) 
#torch.ones(40,device=device)
#target[20:]=-1.0
#target=(yt[0:20]).to(device)
    InsDel=np.empty((nb_iter,2))
    node_costs,nodeInsDel,edge_costs,edgeInsDel=model.from_weighs_to_costs()
    nodeSub=np.empty((nb_iter,int(node_costs.shape[0]*(node_costs.shape[0]-1)/2)))
    edgeSub=np.empty((nb_iter,int(edge_costs.shape[0]*(edge_costs.shape[0]-1)/2)))
    loss_plt=np.empty(nb_iter)
    for t in range(nb_iter):    
        # Forward pass: Compute predicted y by passing data to the model    


        y_pred = model(input).to(device)
    
    
        # Compute and print loss
        loss = criterion(y_pred, target).to(device)    
        node_costs,nodeInsDel,edge_costs,edgeInsDel=model.from_weighs_to_costs()
        triangularInq=criterionTri(node_costs,nodeInsDel,edge_costs,edgeInsDel)
        loss=loss*(1+triangularInq)
        loss.to(device)
        InsDel[t][0]=nodeInsDel.item()
        InsDel[t][1]=edgeInsDel.item()
        loss_plt[t]=loss.item()
        if (loss.item()< min_loss):
            min_loss = loss.item()
            iter_min_loss=t
        print(t, loss.item())  
        print('min_loss = ', min_loss)
        k=0
        for p in range(node_costs.shape[0]):
            for q in range(p+1,node_costs.shape[0]):
                nodeSub[t][k]=node_costs[p][q]
                k=k+1
        k=0
        for p in range(edge_costs.shape[0]):
            for q in range(p+1,edge_costs.shape[0]):
                edgeSub[t][k]=edge_costs[p][q]
                k=k+1
        if t==iter_min_loss: print('node_costs for min of the loss : ', node_costs)
        if t % 100 == 99 or t==0:                         
            print('Distances: ',y_pred)
            print('Loss Triangular:',triangularInq.item())
            print('node_costs :')
            print(node_costs)
            print('nodeInsDel:',nodeInsDel)
            print('edge_costs :')
            print(edge_costs)
            print('edgeInsDel:',edgeInsDel)
            print('min_loss = ', min_loss)
            print('iter_min_loss = ', iter_min_loss)
            
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    nodeInsDel_min = InsDel[iter_min_loss][0]
    edgeInsDel_min = InsDel[iter_min_loss][1]
    nodeSub_min = nodeSub[iter_min_loss]
    edgeSub_min = edgeSub[iter_min_loss]
    print(' Min cost for nodeInsDel = ', nodeInsDel_min)
    print(' Min cost for edgeInsDel = ', edgeInsDel_min)
    print(' Min cost for nodeSub = ', nodeSub_min)
    print(' Min cost for edgeSub = ', edgeSub_min)
    return InsDel,nodeSub,edgeSub,loss_plt

nb_iter=200
InsDel, nodeSub,edgeSub,loss_plt=classification(model,data,yt,nb_iter)
plt.figure(0)
plt.plot(InsDel[0:nb_iter,0],label="node")
plt.plot(InsDel[0:nb_iter,1],label="edge")
plt.title('Node/Edge insertion/deletion costs')
plt.legend()
plt.figure(1)
for k in range(nodeSub.shape[1]):
    plt.plot(nodeSub[0:nb_iter,k])
plt.title('Node Substitutions costs')
plt.figure(2)
for k in range(edgeSub.shape[1]):
    plt.plot(edgeSub[0:nb_iter,k])
plt.title('Edge Substitutions costs')
plt.figure(3)
plt.plot(loss_plt)
plt.title('Evolution of the loss')
plt.show()
plt.close()

cpu
0 0.5307437181472778
min_loss =  0.5307437181472778
node_costs for min of the loss :  tensor([[0.0000, 0.0057, 0.0058, 0.0058, 0.0063, 0.0052, 0.0057, 0.0058, 0.0050,
         0.0052, 0.0064, 0.0060, 0.0055, 0.0055, 0.0051, 0.0066, 0.0050, 0.0057,
         0.0061],
        [0.0057, 0.0000, 0.0051, 0.0052, 0.0049, 0.0060, 0.0056, 0.0064, 0.0058,
         0.0056, 0.0050, 0.0062, 0.0052, 0.0066, 0.0049, 0.0057, 0.0066, 0.0056,
         0.0055],
        [0.0058, 0.0051, 0.0000, 0.0057, 0.0050, 0.0049, 0.0055, 0.0061, 0.0061,
         0.0061, 0.0060, 0.0056, 0.0064, 0.0058, 0.0060, 0.0052, 0.0063, 0.0063,
         0.0054],
        [0.0058, 0.0052, 0.0057, 0.0000, 0.0049, 0.0066, 0.0056, 0.0059, 0.0060,
         0.0054, 0.0061, 0.0053, 0.0053, 0.0052, 0.0050, 0.0054, 0.0065, 0.0052,
         0.0060],
        [0.0063, 0.0049, 0.0050, 0.0049, 0.0000, 0.0067, 0.0055, 0.0050, 0.0067,
         0.0051, 0.0060, 0.0049, 0.0060, 0.0054, 0.0058, 0.0049, 0.0057, 0.0050,
         0.0058],
        [0

1 0.5302354097366333
min_loss =  0.5302354097366333
node_costs for min of the loss :  tensor([[0.0000, 0.0056, 0.0057, 0.0057, 0.0064, 0.0050, 0.0056, 0.0057, 0.0048,
         0.0051, 0.0065, 0.0059, 0.0053, 0.0053, 0.0050, 0.0067, 0.0048, 0.0057,
         0.0060],
        [0.0056, 0.0000, 0.0049, 0.0050, 0.0047, 0.0060, 0.0055, 0.0065, 0.0057,
         0.0055, 0.0048, 0.0062, 0.0050, 0.0067, 0.0047, 0.0056, 0.0067, 0.0055,
         0.0054],
        [0.0057, 0.0049, 0.0000, 0.0056, 0.0047, 0.0047, 0.0054, 0.0061, 0.0061,
         0.0061, 0.0059, 0.0055, 0.0064, 0.0058, 0.0059, 0.0050, 0.0063, 0.0063,
         0.0053],
        [0.0057, 0.0050, 0.0056, 0.0000, 0.0047, 0.0066, 0.0055, 0.0059, 0.0060,
         0.0052, 0.0061, 0.0052, 0.0051, 0.0050, 0.0048, 0.0052, 0.0065, 0.0050,
         0.0060],
        [0.0064, 0.0047, 0.0047, 0.0047, 0.0000, 0.0067, 0.0054, 0.0048, 0.0068,
         0.0050, 0.0060, 0.0046, 0.0060, 0.0053, 0.0058, 0.0047, 0.0056, 0.0048,
         0.0058],
        [0.005

4 0.5295997262001038
min_loss =  0.5295997262001038
node_costs for min of the loss :  tensor([[0.0000, 0.0044, 0.0046, 0.0046, 0.0057, 0.0035, 0.0044, 0.0046, 0.0031,
         0.0035, 0.0059, 0.0049, 0.0039, 0.0039, 0.0034, 0.0062, 0.0032, 0.0045,
         0.0051],
        [0.0044, 0.0000, 0.0033, 0.0034, 0.0029, 0.0051, 0.0042, 0.0059, 0.0045,
         0.0042, 0.0032, 0.0054, 0.0034, 0.0063, 0.0030, 0.0044, 0.0062, 0.0042,
         0.0040],
        [0.0046, 0.0033, 0.0000, 0.0045, 0.0030, 0.0030, 0.0040, 0.0052, 0.0052,
         0.0053, 0.0049, 0.0043, 0.0058, 0.0047, 0.0050, 0.0035, 0.0056, 0.0055,
         0.0039],
        [0.0046, 0.0034, 0.0045, 0.0000, 0.0029, 0.0062, 0.0043, 0.0049, 0.0050,
         0.0038, 0.0052, 0.0037, 0.0036, 0.0034, 0.0032, 0.0038, 0.0060, 0.0034,
         0.0050],
        [0.0057, 0.0029, 0.0030, 0.0029, 0.0000, 0.0064, 0.0041, 0.0031, 0.0064,
         0.0034, 0.0050, 0.0029, 0.0051, 0.0039, 0.0047, 0.0030, 0.0043, 0.0031,
         0.0047],
        [0.003

7 0.5294504761695862
min_loss =  0.5294504761695862
node_costs for min of the loss :  tensor([[0.0000, 0.0010, 0.0012, 0.0012, 0.0024, 0.0004, 0.0010, 0.0012, 0.0003,
         0.0005, 0.0026, 0.0015, 0.0007, 0.0007, 0.0004, 0.0029, 0.0003, 0.0011,
         0.0016],
        [0.0010, 0.0000, 0.0003, 0.0004, 0.0002, 0.0016, 0.0009, 0.0026, 0.0011,
         0.0009, 0.0003, 0.0018, 0.0004, 0.0031, 0.0002, 0.0010, 0.0029, 0.0009,
         0.0007],
        [0.0012, 0.0003, 0.0000, 0.0011, 0.0002, 0.0002, 0.0007, 0.0017, 0.0017,
         0.0018, 0.0015, 0.0009, 0.0025, 0.0012, 0.0015, 0.0004, 0.0023, 0.0020,
         0.0007],
        [0.0012, 0.0004, 0.0011, 0.0000, 0.0002, 0.0029, 0.0010, 0.0014, 0.0015,
         0.0006, 0.0017, 0.0006, 0.0005, 0.0004, 0.0003, 0.0006, 0.0027, 0.0004,
         0.0015],
        [0.0024, 0.0002, 0.0002, 0.0002, 0.0000, 0.0031, 0.0008, 0.0002, 0.0032,
         0.0004, 0.0015, 0.0002, 0.0016, 0.0007, 0.0013, 0.0002, 0.0010, 0.0003,
         0.0012],
        [0.000

9 0.5293681025505066
min_loss =  0.5293681025505066
node_costs for min of the loss :  tensor([[0.0000e+00, 2.9183e-06, 1.4222e-06, 8.4572e-07, 3.6989e-04, 1.5730e-04,
         1.5952e-06, 3.5623e-07, 2.7777e-04, 1.3896e-04, 4.5536e-04, 1.8798e-05,
         4.7447e-05, 4.7510e-05, 1.8808e-04, 5.8586e-04, 2.6247e-04, 3.0094e-07,
         4.3517e-05],
        [2.9183e-06, 0.0000e+00, 2.1577e-04, 1.8383e-04, 3.7044e-04, 3.3802e-05,
         1.3129e-05, 4.3274e-04, 2.9705e-08, 1.3048e-05, 2.7182e-04, 7.8282e-05,
         1.8312e-04, 6.3705e-04, 3.6168e-04, 2.9664e-06, 5.8622e-04, 1.1892e-05,
         3.8440e-05],
        [1.4222e-06, 2.1577e-04, 0.0000e+00, 7.4318e-07, 3.2556e-04, 3.6662e-04,
         3.9567e-05, 5.2893e-05, 4.8571e-05, 6.8265e-05, 1.8588e-05, 9.1616e-06,
         3.8826e-04, 2.2057e-06, 2.1970e-05, 1.4649e-04, 3.3247e-04, 1.1119e-04,
         5.0953e-05],
        [8.4572e-07, 1.8383e-04, 7.4318e-07, 0.0000e+00, 3.8792e-04, 5.7711e-04,
         6.6852e-06, 1.3031e-05, 2.776

11 0.5409431457519531
min_loss =  0.5293187499046326
12 0.5296047925949097
min_loss =  0.5293187499046326
13 0.5294532179832458
min_loss =  0.5293187499046326
14 0.5294163823127747
min_loss =  0.5293187499046326
15 0.5294631123542786
min_loss =  0.5293187499046326
16 0.5295045971870422
min_loss =  0.5293187499046326
17 0.5295407772064209
min_loss =  0.5293187499046326
18 0.5295719504356384
min_loss =  0.5293187499046326
19 0.5295981764793396
min_loss =  0.5293187499046326
20 0.5296202301979065
min_loss =  0.5293187499046326
21 0.5296381115913391
min_loss =  0.5293187499046326
22 0.5296524167060852
min_loss =  0.5293187499046326
23 0.5296634435653687
min_loss =  0.5293187499046326
24 0.5296717286109924
min_loss =  0.5293187499046326
25 0.5296773314476013
min_loss =  0.5293187499046326
26 0.5296807885169983
min_loss =  0.5293187499046326
27 0.5296822786331177
min_loss =  0.5293187499046326
28 0.5296820998191833
min_loss =  0.5293187499046326
29 0.5296804904937744
min_loss =  0.5293187499

100 0.5295639634132385
min_loss =  0.5293187499046326
101 0.5295643210411072
min_loss =  0.5293187499046326
102 0.5295654535293579
min_loss =  0.5293187499046326
103 0.5295692086219788
min_loss =  0.5293187499046326
104 0.5295679569244385
min_loss =  0.5293187499046326
105 0.5295689702033997
min_loss =  0.5293187499046326
106 0.5295735597610474
min_loss =  0.5293187499046326
107 0.5295698046684265
min_loss =  0.5293187499046326
108 0.5295696258544922
min_loss =  0.5293187499046326
109 0.5295686721801758
min_loss =  0.5293187499046326
110 0.529567301273346
min_loss =  0.5293187499046326
111 0.5295652747154236
min_loss =  0.5293187499046326
112 0.5295681953430176
min_loss =  0.5293187499046326
113 0.5295624732971191
min_loss =  0.5293187499046326
114 0.5295615792274475
min_loss =  0.5293187499046326
115 0.5295600891113281
min_loss =  0.5293187499046326
116 0.5295580625534058
min_loss =  0.5293187499046326
117 0.5295556783676147
min_loss =  0.5293187499046326
118 0.5295528769493103
min_lo

 Min cost for nodeInsDel =  0.001308500417508185
 Min cost for edgeInsDel =  0.19402042031288147
 Min cost for nodeSub =  [2.14166837e-04 1.42646386e-04 1.48759311e-04 4.20803626e-05
 6.08724775e-04 2.02249314e-04 1.56154172e-04 8.11594306e-04
 5.74781501e-04 7.16024515e-05 8.14769955e-05 3.77772842e-04
 3.77935445e-04 6.63441606e-04 1.24279963e-04 7.87247904e-04
 1.83840268e-04 4.79858318e-05 7.10726250e-04 6.56033866e-04
 9.53200448e-04 5.85842026e-05 2.69298092e-04 6.33396776e-05
 1.74497996e-04 2.68958916e-04 8.02160997e-04 2.33936044e-05
 6.54797477e-04 1.46858729e-04 9.40179103e-04 2.14547385e-04
 1.24432408e-04 2.63999886e-04 3.53387906e-04 1.91810468e-04
 8.85742716e-04 9.47525667e-04 3.56538862e-04 3.96826363e-05
 4.33124806e-05 2.89492182e-05 8.18835179e-05 2.51449004e-04
 4.80132730e-05 1.36234856e-04 7.56908848e-05 5.88864554e-04
 3.08071139e-05 1.06299321e-05 3.86827363e-04 9.78986965e-04
 1.20517594e-04 2.38640612e-04 9.40931714e-05 6.65831612e-05
 4.31350549e-04 4.260533

In [8]:
A=torch.tensor(nx.to_scipy_sparse_matrix(Gs[0],dtype=int,weight='bond_type').todense(),dtype=torch.int) 
print(A)

tensor([[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0],
        [0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0],
        [1, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]], dtype=torch.int32)


In [9]:
plt.figure(0)
plt.plot(InsDel[0:500,0],label="node")
plt.plot(InsDel[0:500,1],label="edge")
plt.title('Node/Edge insertion/deletion costs')
plt.legend()
plt.figure(1)
for k in range(nodeSub.shape[1]):
    plt.plot(nodeSub[0:500,k])
plt.title('node Substitution costs')
plt.figure(2)
for k in range(edgeSub.shape[1]):
    plt.plot(edgeSub[0:500,k])
plt.title('edge Substitution costs')
plt.show()
plt.close()